In [7]:
import numpy as np
import corona as cr
from glob import glob
from astropy.io import fits
import matplotlib.pyplot as plt
import os
import shutil
from gootools import progress_bar
from kirdap import irdap

irdap_main_path = "D:/Science/LBV/test_irdap"

---
### Make the necessary file structure
---

In [8]:
#make one main_path for each target
target_paths = glob(irdap_main_path+'/*')
for t in cr.targets:
    dir_exists = False
    for p in target_paths:
        if t in p:
            dir_exists = True
    if not dir_exists:
        os.makedirs(irdap_main_path+'/'+t)
        
#make sure every main path has a raw/ directory and has a config file
for p in target_paths:
    if p+'\\raw' in glob(p+'/*'):
        #copy config file for irdap
        shutil.copy('kirdap/config.conf',p+'/config.conf')
        continue
    else:
        os.mkdir(p+'/raw')
        #copy config file for irdap
        shutil.copy('kirdap/config.conf',p+'/config.conf')

In [9]:
#Here we need to figure out which files to include and fix 'em
for p in target_paths:
    print('\nCopying files for {0}'.format(target := p.split('\\')[-1])) #ayyyy i figured out how to use the walrus!
    file_paths = np.array([f    for f in cr.files if target in f])
    #We only want to copy over object files with the same exposure time
    object_file_paths = np.array([f for f in file_paths if 'OBJECT' in f and any([cr.header(f)['ESO INS4 FILT2 NAME']=='OPEN','FLUX' in f])]) #this includes center and flux files   
    #now we need to make sure that there are the same number of darks and flats (for some reason)
    dark_file_paths = np.array([f for f in file_paths if 'DARK' in f])
    flat_file_paths = np.array([f for f in file_paths if 'FLAT' in f])
    sky_file_paths = np.array([f for f in file_paths if 'SKY' in f and f.split('\\')[-1] not in cr.sat_files])
    while len(dark_file_paths) > len(flat_file_paths):
        dark_file_paths = np.delete(dark_file_paths,-1)
        
    #collect all of em
    home_paths = np.r_[\
                      object_file_paths,
                      dark_file_paths,
                      flat_file_paths,
#                       sky_file_paths
                     ]
    for i,f in enumerate(home_paths):
        if p+'/raw\\'+(file_name := f.split('\\')[-1]) not in glob(p+'/raw/*'):
            with fits.open(f,output_verify='ignore') as hdul:
                exposure_time = hdul[0].header['ESO DET SEQ1 DIT'] #exposure time in seconds
                data = hdul[0].data.copy()
                new_header = hdul[0].header.copy()
                if hdul[0].header['ESO DPR CATG'] == 'ACQUISITION':
                    new_header['ESO DPR CATG'] = 'CALIB'
                    new_header['ESO INS1 FILT ID'] = 'FILT_BBF_H'
                del new_header['ESO INS1 SENS101 VAL']
                del new_header['ESO INS4 SENS424 VAL']
                if not any([f in flat_file_paths,f in dark_file_paths]):
                    data /= exposure_time
                    new_header['ESO DET SEQ1 DIT'] = 1.0
                fits.writeto(p+'/raw/'+file_name,data,header=new_header,overwrite=True)
        progress_bar(i+1,len(home_paths))



Copying files for AS-314
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for HD-160529
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for HD-168607
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for HD-168625
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for HD-316285
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for HD-326823
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for MWC-314
|████████████████████████████████████████████████████████████████████████████████████████████████████|100.0%
Copying files for ZETA-SCO
|█████████████████

---
### Run IRDAP
---

In [10]:
for path in target_paths:
    if len(glob(path+'\\raw/*FLAT*.fits')) > 1.:
        irdap.run_pipeline(path)


Starting data reduction.

###############################################################################
# Important notice
###############################################################################

When publishing data reduced with IRDAP, please cite van Holstein et al.
(2020): https://ui.adsabs.harvard.edu/abs/2020A%26A...633A..64V/abstract.
For data in pupil-tracking mode please additionally cite van Holstein et al.
(2017): http://adsabs.harvard.edu/abs/2017SPIE10400E..15V.

Full documentation: https://irdap.readthedocs.io
Feedback, questions, comments: vanholstein@strw.leidenuniv.nl

IRDAP Copyright (C) 2019 R.G. van Holstein

###############################################################################
# Starting IRDAP
###############################################################################

Running IRDAP version 1.3.2

Date: 2021-04-16
Time: 17:51:20

###############################################################################
# Preparing data reduction
######

---